# Generate a list of boundary-plane vectors

In this code, we determine the list of boundary-plane orientation vectors, represented using the Miller indices in the primitve **Coincidence Site Lattice**. These boundary-plane vectors  fall in the fundamental zone of boundary-plane orientations.

1. Load relevant modules
2. Upload necessary files
    + `pkl_name`: The pkl file which contains the bicrystallography properties for each Σ-miorientation.
3. Compute indices of boundary-plane orientation for a given Σ-misorientation
4. Save the list of boundary-plane orientations in hkl_sig_inds dictionary.

In [2]:
import byxtal.tools as gbt
import byxtal.lattice as gbl
import pickle as pkl
import numpy as np
import numpy.linalg as nla
import byxtal.misorient_fz as mfz
import gbpy.generate_hkl_indices as ghi

1. `l1`: The lattice object.
2. `l_p_po`: $\Lambda_p^{po}$
3. `sig_mats`: The $\Sigma$-miorientation in the primitive $p$ reference frame. For example, we use the variable `T_p1top2_p1`
4. `csl_mats`: The CSL lattice basis vectors in the primitive $p$ reference frame. For example, we use the variable `l_csl_p` denoting $\Lambda_{csl}^p$
5. `dsc_mats`: The DSC lattice basis vectors in the primitive $p$ reference frame. For example, we use the variable `l_dsc_p` denoting $\Lambda_{dsc}^p$
6. `bxt_symm_props`: The symmetry aspects of the $\Sigma$-miorientation
    + `bxt_symm_grp`: The point-group symmetry of the bicrystal. This is the symmetry for the Boundary-plane orientations.
    + `symm_grp_ax`: The axes of the symmetry point-group.
7. `num1`: Refinement of the grid on the 2-sphere for determining boundary-plane orientations. For example, `num1=3` will produce approximately (of the order of) `num1^3 (=27)` vectors.
    + The number of boundary-plane orientations will depend on the bicrystal symmetry of the $\Sigma$ misorientation.

In [3]:
######################################################################
###### Boundary-plane matrix properties with A_cut
#####################################################################
l1=gbl.Lattice('Mg_sun')
pkl_name = l1.elem_type + '_byxtal_props.pkl'
jar = open(pkl_name,'rb')
s1=pkl.load(jar)
jar.close()

sig_mats = s1['sig_mats']
csl_mats = s1['csl_mats']
dsc_mats = s1['dsc_mats']
bxt_symm_props = s1['csl_symm_props']

l_p_po = l1.l_p_po

num1 = 3

1. `s1_keys`: The "string" keys for the variables `sig_mats`, `csl_mats`, `dsc_mats`, `bxt_symm_props`
2. `hkl_sig_inds`: Dictionary for saving boundary-plane orientations for each $\Sigma$-misorientation
    + `hkl_inds`: Indices of boundary-plane orientation for a given $\Sigma$-misorientation

In [4]:
s1_keys = list(bxt_symm_props.keys())
hkl_sig_inds = {}

for sig_id in s1_keys:
    print(sig_id)

    l_p_po = l1.l_p_po
    l_po_p = nla.inv(l_p_po)

    bp_symm_grp = bxt_symm_props[sig_id]['bxt_symm_grp']
    symm_grp_ax = bxt_symm_props[sig_id]['symm_grp_ax']
    l_csl_p = csl_mats[sig_id]


    l_csl_po = l_p_po.dot(l_csl_p)
    l_csl_props = {}

    l_csl_props['l_csl_po'] = l_csl_po
    l_csl_props['symm_grp_ax'] = symm_grp_ax
    l_csl_props['bp_symm_grp'] = bp_symm_grp

    hkl_inds = ghi.gen_hkl_props(l_csl_props, num1)
    hkl_sig_inds[sig_id] = hkl_inds
    

7_1
13_1


Save the list of boundary-plane orientations in `hkl_sig_inds` dictionary.

In [5]:
########################################################################
pkl_name = 'bp_list_' + l1.elem_type + '.pkl'
jar = open(pkl_name, 'wb')
bp_list = {}
bp_list['l_p_po'] = l_p_po
bp_list['miller_inds'] = hkl_sig_inds
pkl.dump(bp_list, jar)
jar.close()
########################################################################
